<a href="https://colab.research.google.com/github/datafyi/datascience.fyi/blob/main/extra_tree_classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [68]:
import pandas as pd

cancer_data = pd.read_csv("cancer_dataset.csv")

In [69]:
# cancer_data = cancer_data.drop(columns=['id'])

In [70]:
cancer_data.diagnosis = cancer_data.diagnosis.map({'M':1,'B':0})

In [71]:
cancer_data = cancer_data.drop('Unnamed: 32', axis=1)
cancer_data.head()

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
0,842302,1,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,...,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,842517,1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,...,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,84300903,1,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,...,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
3,84348301,1,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,...,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
4,84358402,1,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,...,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678


In [72]:
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.datasets import make_classification

In [73]:
X = cancer_data.drop(columns=['diagnosis'])
y = cancer_data.diagnosis

In [74]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1)

In [75]:
numeric_features = X.columns
# categorical_features = []

In [76]:
from sklearn.compose import ColumnTransformer 
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder

In [77]:
drop_feat=['id']
drop_transformer = ColumnTransformer(transformers=[('drop_columns', 'drop', drop_feat)], remainder='passthrough')

In [78]:
from sklearn.pipeline import Pipeline

In [79]:
pipeline = Pipeline([('drop_column', drop_transformer)])

In [80]:
pipeline.fit(X_train)

Pipeline(steps=[('drop_column',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('drop_columns', 'drop',
                                                  ['id'])]))])

In [82]:
transformed_train=pipeline.transform(X_train)
transformed_train

array([[1.152e+01, 1.493e+01, 7.387e+01, ..., 9.608e-02, 2.664e-01,
        7.809e-02],
       [1.305e+01, 1.931e+01, 8.261e+01, ..., 1.111e-02, 2.439e-01,
        6.289e-02],
       [1.420e+01, 2.053e+01, 9.241e+01, ..., 1.339e-01, 2.534e-01,
        7.858e-02],
       ...,
       [1.720e+01, 2.452e+01, 1.142e+02, ..., 1.899e-01, 3.313e-01,
        1.339e-01],
       [1.403e+01, 2.125e+01, 8.979e+01, ..., 7.963e-02, 2.226e-01,
        7.617e-02],
       [1.303e+01, 1.842e+01, 8.261e+01, ..., 5.013e-02, 1.987e-01,
        6.169e-02]])

In [83]:
numeric_transformer = Pipeline(steps=[
                                     ('meanimputer', SimpleImputer(strategy='mean')),
                                     ('stdscaler', StandardScaler())
                                     ])

In [84]:
# categorical_transformer = Pipeline(steps=[
#                                          ('onehotenc', OneHotEncoder(handle_unknown='ignore'))
#                                          ])

In [85]:
col_transformer = ColumnTransformer(transformers=[('drop_columns', 'drop', drop_feat),
                                                   ('numeric_processing',numeric_transformer, numeric_features)
                                                   # ,('categorical_processing', categorical_transformer, categorical_features)
                                                  ], remainder='drop')

In [86]:
pipeline = Pipeline([('transform_column', col_transformer)])

In [87]:
pipeline.fit(X_train)

Pipeline(steps=[('transform_column',
                 ColumnTransformer(transformers=[('drop_columns', 'drop',
                                                  ['id']),
                                                 ('numeric_processing',
                                                  Pipeline(steps=[('meanimputer',
                                                                   SimpleImputer()),
                                                                  ('stdscaler',
                                                                   StandardScaler())]),
                                                  Index(['id', 'radius_mean', 'texture_mean', 'perimeter_mean', 'area_mean',
       'smoothness_mean', 'compactness_mean', 'concavity_mean',
       'concave points_mean', 'sy...
       'radius_se', 'texture_se', 'perimeter_se', 'area_se', 'smoothness_se',
       'compactness_se', 'concavity_se', 'concave points_se', 'symmetry_se',
       'fractal_dimension_se', 'radius_wor

In [88]:
transformed_train=pipeline.transform(X_train)

In [89]:
pd.DataFrame(transformed_train)

,0,1,2,3,4,5,6,7,8,9,...,21,22,23,24,25,26,27,28,29,30
0,-0.252447,-0.718835,-1.012704,-0.724558,-0.690529,0.342745,-0.528382,-0.572826,-0.506824,0.277754,...,-0.729777,-0.712004,-0.766628,-0.670318,0.287945,-0.637009,-0.448343,-0.281546,-0.358810,-0.346080
1,-0.252630,-0.302167,-0.019346,-0.379219,-0.363096,-1.144967,-1.311635,-1.104054,-1.142451,0.043414,...,-0.418576,-0.543809,-0.500781,-0.451605,-1.349057,-1.265020,-1.340965,-1.548733,-0.756728,-1.248786
2,-0.252265,0.011015,0.257343,0.008003,-0.116099,-0.518978,0.109290,-0.481144,-0.474186,-1.102653,...,0.018680,0.251148,0.120095,-0.113698,-0.761871,0.567618,-0.094404,0.282477,-0.588718,-0.316980
3,-0.252219,-0.569052,-1.350628,-0.568088,-0.585177,0.968015,-0.294322,-0.641057,-0.539967,-0.549758,...,-0.572207,-1.572017,-0.595077,-0.574270,0.875130,-0.604399,-0.539327,-0.611876,-0.151893,-0.201766
4,-0.252299,-0.694325,-0.765499,-0.661733,-0.685384,0.881771,0.226223,-0.227430,-0.110110,0.288739,...,-0.731746,-0.953188,-0.736805,-0.696934,0.403603,-0.253511,-0.199386,-0.066495,-0.141282,-0.572944
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
393,-0.252570,1.533351,1.298328,1.509080,1.437382,0.357119,1.046700,2.061860,1.659162,1.445790,...,1.235906,1.253967,1.159623,1.143535,-0.219170,0.849371,1.485825,0.868571,0.774813,0.043509
394,-0.252552,-0.928530,-1.003633,-0.946222,-0.828653,-1.336861,-1.048537,-0.831904,-1.048636,-1.494439,...,-0.867650,-0.786581,-0.854391,-0.753969,-1.104396,-0.871152,-0.862522,-1.205428,-1.002552,-0.963721
395,-0.252617,0.828011,1.162252,0.868979,0.726182,0.759592,1.516379,0.997857,0.762002,0.438862,...,1.371810,1.292049,1.241989,1.295626,1.159826,3.153620,1.932249,1.117624,0.788961,2.968394
396,0.339610,-0.035281,0.420635,-0.095520,-0.156723,-0.419079,-0.696570,-0.930321,-0.768179,-1.062376,...,-0.201918,0.730343,-0.272710,-0.300505,-0.165789,-0.682012,-1.038692,-0.526870,-1.133423,-0.460106


In [90]:
pipeline.transform(X_test)

array([[-0.25229919,  0.14445772, -1.22815887, ..., -0.06202148,
        -0.07054093,  0.48476556],
       [-0.25786194, -0.26948682, -0.1667621 , ...,  1.39948615,
         1.82708337,  2.01817732],
       [-0.25240012, -0.32939985, -0.76549874, ..., -0.13957087,
         0.90745184,  0.70806642],
       ...,
       [ 0.31494486,  0.44129954, -0.34139362, ...,  0.98489521,
         1.63608298,  1.24018762],
       [-0.19805783, -0.3511864 , -0.72240785, ..., -0.85242867,
         1.30360081, -0.78971222],
       [-0.25230104, -0.81687402,  0.45692189, ..., -1.07225135,
         0.19296888, -0.75348522]])

In [91]:
pipeline = Pipeline([
                     ('transform_column', col_transformer),
                     ('logistics', ExtraTreesClassifier())
                    ])

In [92]:
pipeline.fit(X_train, y_train)

Pipeline(steps=[('transform_column',
                 ColumnTransformer(transformers=[('drop_columns', 'drop',
                                                  ['id']),
                                                 ('numeric_processing',
                                                  Pipeline(steps=[('meanimputer',
                                                                   SimpleImputer()),
                                                                  ('stdscaler',
                                                                   StandardScaler())]),
                                                  Index(['id', 'radius_mean', 'texture_mean', 'perimeter_mean', 'area_mean',
       'smoothness_mean', 'compactness_mean', 'concavity_mean',
       'concave points_mean', 'sy...
       'radius_se', 'texture_se', 'perimeter_se', 'area_se', 'smoothness_se',
       'compactness_se', 'concavity_se', 'concave points_se', 'symmetry_se',
       'fractal_dimension_se', 'radius_wor

In [94]:
pipeline.score(X_test, y_test)

0.9532163742690059